# Eseguire la distribuzione in un endpoint online

Per utilizzare un modello da un'applicazione, è possibile distribuire il modello in un endpoint online. Si creerà un modello MLflow dai file locali e si testerà l'endpoint.

## Prima di iniziare

Per eseguire il codice in questo notebook, è necessaria la versione più recente del pacchetto  **azureml-ai-ml** . Eseguire la cella seguente per verificare che sia installata.

> **Nota**:
> Se il pacchetto **azure-ai-ml** non è installato, eseguire `pip install azure-ai-ml` per installarlo.

In [ ]:
## Connettersi all'area di lavoro

Dopo aver installato i pacchetti SDK necessari, è ora possibile connettersi all'area di lavoro.

Per connettersi a un'area di lavoro, sono necessari parametri di identificatore: ID sottoscrizione, nome del gruppo di risorse e nome dell'area di lavoro. Il nome del gruppo di risorse e il nome dell'area di lavoro sono già compilati automaticamente. Per completare il comando è necessario solo l'ID sottoscrizione.

Per trovare i parametri necessari, fare clic sulla sottoscrizione e sul nome dell'area di lavoro in alto a destra di Studio. Verrà aperto un riquadro a destra.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiare l'ID sottoscrizione e sostituire **YOUR-SUBSCRIPTION-ID** con il valore copiato. </p>

## Definire e creare un endpoint

Infine, l'obiettivo è distribuire un modello in un endpoint. Pertanto, è prima necessario creare un endpoint. L'endpoint sarà un endpoint HTTPS che un'applicazione può chiamare per ricevere stime dal modello. Un'applicazione può utilizzare un endpoint usando il relativo URI e autenticando con una chiave o un token.

Eseguire la cella seguente per definire l'endpoint. Si noti che il nome dell'endpoint deve essere univoco. Si userà la `datetime` funzione per generare un nome univoco.

In [ ]:
Successivamente, si creerà l'endpoint eseguendo la cella seguente. Questa operazione può richiedere alcuni minuti. Durante la creazione dell'endpoint, è possibile leggere le informazioni sugli [endpoint di Azure Machine Learning](https://learn.microsoft.com/azure/machine-learning/concept-endpoints).

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE Attendere che l'endpoint venga creato correttamente prima di continuare. Verrà visualizzata una notifica verde nello studio. </p>

## Configurare la distribuzione

È possibile distribuire più modelli in un endpoint. Ciò è particolarmente utile quando si vuole aggiornare il modello distribuito mantenendo il modello corrente nell'ambiente di produzione. È necessario configurare la distribuzione per specificare il modello da distribuire in un endpoint. Nella cella seguente si farà riferimento al modello sottoposto a training e archiviato nella cartella locale `model` ,archiviata nella stessa cartella del notebook. Si noti che, poiché si usa un modello MLflow, non è necessario specificare l'ambiente o lo script di assegnazione dei punteggi.

Specificare anche l'infrastruttura necessaria per la distribuzione del modello.

## Creare la distribuzione

Infine, è possibile distribuire effettivamente il modello nell'endpoint eseguendo la cella seguente:

In [ ]:
La distribuzione del modello può richiedere da 10 a 15 minuti. In attesa della distribuzione del modello, è possibile ottenere altre informazioni sugli [endpoint gestiti in questo video](https://www.youtube.com/watch?v=SxFGw_OBxNM&ab_channel=MicrosoftDeveloper).

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE Attendere il completamento della distribuzione prima di continuare. Verrà visualizzata una notifica verde nello studio.</p>

Poiché nell'endpoint è stato distribuito un solo modello, si vuole che questa distribuzione assuma il 100% del traffico. Se si distribuiscono più modelli nell'endpoint, è possibile usare lo stesso approccio per distribuire il traffico tra i modelli distribuiti.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANTE Attendere che la distribuzione blu sia configurata prima di continuare. Verrà visualizzata una notifica verde nello studio. </p> 

## Test della distribuzione

Testare il modello distribuito richiamando l'endpoint. Un file JSON con dati di esempio viene usato come input. Il modello sottoposto a training consente di stimare se un paziente ha o meno il diabete, in base a dati medici come età, BMI e numero di gravidanze. Un `[0]` indica che un paziente non ha il diabete. Un `[1]` significa che un paziente ha il diabete.

In [ ]:
Facoltativamente, è possibile modificare i valori nel `sample-data.json` file per provare a ottenere una stima diversa.

## Elencare gli endpoint

Sebbene sia possibile visualizzare tutti gli endpoint in Studio, è anche possibile elencare tutti gli endpoint usando l'SDK:

In [ ]:
## Ottenere i dettagli dell'endpoint

Per altre informazioni su un endpoint specifico, è possibile esplorare anche i dettagli usando l'SDK.

## Eliminare l'endpoint e la distribuzione

Poiché un endpoint è sempre disponibile, non può essere sospeso per risparmiare i costi. Per evitare costi non necessari, eliminare l'endpoint.

In [ ]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

The deployment of the model may take 10-15 minutes. While waiting for the model to be deployed, you can learn more about [managed endpoints in this video](https://www.youtube.com/watch?v=SxFGw_OBxNM&ab_channel=MicrosoftDeveloper).

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio.</p>

Since you only have one model deployed to the endpoint, you want this deployment to take 100% of the traffic. If you deploy multiple models to the endpoint, you could use the same approach to distribute traffic across the deployed models.

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the blue deployment is configured before continuing! A green notification should appear in the studio. </p> 

## Test the deployment

Let's test the deployed model by invoking the endpoint. A JSON file with sample data is used as input. The trained model predicts whether a patient has diabetes or not, based on medical data like age, BMI, and the number of pregnancies. A `[0]` indicates a patient doesn't have diabetes. A `[1]` means a patient does have diabetes.

In [ ]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample-data.json",
)

if response[1]=='1':
    print("Diabetic")
else:
    print ("Not diabetic")

Optionally, you can change the values in the `sample-data.json` file to try and get a different prediction.

## List endpoints

Although you can view all endpoints in the Studio, you can also list all endpoints using the SDK:

In [ ]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

## Get endpoint details

If you want more information about a specific endpoint, you can explore the details using the SDK too.

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## Delete the endpoint and deployment

As an endpoint is always available, it can't be paused to save costs. To avoid unnecessary costs, delete the endpoint.

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)